### **Data Reading**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format("parquet")\
  .load("abfss://bronze@retailsalesanalysissanil.dfs.core.windows.net/orders")
df.display()

In [0]:
df = df.withColumn("year", year(col("order_date")))
df.display()

In [0]:
window_spec = Window.partitionBy("year").orderBy(col("total_amount").desc())
df1 = df.withColumn("dense_flag", dense_rank().over(window_spec))
df1.display()

In [0]:
df1 = df.withColumn("rank_flag", rank().over(Window.partitionBy(col("year")).orderBy(col("total_amount").desc())))
df1.display()

In [0]:
df1 = df.withColumn("row_flag", row_number().over(Window.partitionBy(col("year")).orderBy(col("total_amount").desc())))
df1.show()

### **Classes - OOP**

In [0]:
class Windows:
  def dense_rank(self, df):
    df_dense_rank = df.withColumn("dense_flag", dense_rank().over(Window.partitionBy(col("year")).orderBy(col("total_amount").desc())))
    return df_dense_rank
  
  def rank(self, df):
    df_rank = df.withColumn("rank_flag", rank().over(Window.partitionBy(col("year")).orderBy(col("total_amount").desc())))
    return df_rank
  
  def row_number(self, df):
    df_row_number = df.withColumn("row_flag", row_number().over(Window.partitionBy(col("year")).orderBy(col("total_number").desc())))
    return df_row_number

In [0]:
df_new = df

In [0]:
df_new.display()

In [0]:
obj = Windows()

In [0]:
df_result = obj.dense_rank(df_new)
df_result.display()

### **Data Writing**

In [0]:
df.write.format("delta")\
  .mode("append")\
  .save("abfss://silver@retailsalesanalysissanil.dfs.core.windows.net/orders")

### **Table Creation to top of the Data**

In [0]:
%sql
create table if not exists rsa_cata.silver.orders_silver
using delta
location 'abfss://silver@retailsalesanalysissanil.dfs.core.windows.net/orders'